In [25]:
import json
import collections

In [26]:
# 读入保存的词表
with open('base_word_dict/processed_word_dict.json', 'r') as f:
    word_dict = json.load(f)

# 定义词表
l_vocab = []

# 添加[PAD],10个[unused],[UNK],[CLS],[SEP],[MASK]
l_vocab.append('[PAD]')
for i in range(10):
    l_vocab.append('[unused'+str(i+1)+']')
    
extra_words = ['[UNK]', '[CLS]', '[SEP]', '[MASK]']
l_vocab = l_vocab + extra_words

# 将词表转换为list，并进行排序
l_words = []
for word, num in word_dict.items():
    l_words.append(word)
l_words.sort() 
l_vocab = l_vocab + l_words

# 保存为vocab
with open('vocab.txt', 'w') as writer:
    for word in l_vocab:
        writer.write(word+'\n')

In [27]:
# 读取vocab
def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    with open(vocab_file, "r", encoding="utf-8") as reader:
        tokens = reader.readlines()
    for index, token in enumerate(tokens):
        token = token.rstrip("\n")
        vocab[token] = index
    return vocab


vocab_file = 'vocab.txt'
vocab = load_vocab(vocab_file)

#### 下面是一些tokenizer的尝试

In [8]:
split = ['黑色', '羊毛衫', '加绒']
input_ids = []
for word in split:
    input_ids.append(vocab[word])
print(input_ids)

[439, 340, 67]


In [22]:
splits = [['休闲', '百搭', '标准型', '男士', '棉服', '黑色', '连帽', '常规款', '男装'], ['加厚', '女装', '白', '鸭绒', '小脚裤', '冬季', '加绒', '裤'], ['哈伦裤', '蓝色', '牛仔裤', '长裤', '加绒', '男装']]
split_length = []
for split in splits:
    split_length.append(len(split))
max_length = max(split_length)

input_ids = []
for split in splits:
    split_ids = []
    for word in split:
        split_ids.append(vocab[word])
    if len(split_ids) < max_length:
        pad = [vocab['[PAD]']] * (max_length - len(split_ids))
        split_ids.extend(pad)
    input_ids.append(split_ids)

In [39]:
import torch 

splits = [['休闲', '百搭', '标准型', '男士', '棉服', '黑色', '连帽', '常规款', '男装'], ['加厚', '女装', '白', '鸭绒', '小脚裤', '冬季', '加绒', '裤'], ['哈伦裤', '蓝色', '牛仔裤', '长裤', '加绒', '男装']]
split_length = []
for split in splits:
    split_length.append(len(split))
max_length = max(split_length)

B = len(splits)
W = max_length
input_ids = torch.zeros(B, W, dtype=int)
mask = torch.zeros(B, W, dtype=int)
for i, split in enumerate(splits):
    for j, word in enumerate(split):
        input_ids[i, j] = vocab[word]
        mask[i, j] = 1